In [44]:
#import necessary libraries
import numpy as np
import pandas as pd

import nltk

#Tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

In [64]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

## Retrieving Data

In [5]:
#Read csv
train = pd.read_csv (r'train.csv')
test = pd.read_csv (r'test.csv')

In [48]:
df_train = pd.DataFrame(train, columns= ['id','raw_address','POI/street'])
print(df_train)

            id                                        raw_address  \
0            0  jl kapuk timur delta sili iii lippo cika 11 a ...   
1            1                                 aye, jati sampurna   
2            2               setu siung 119 rt 5 1 13880 cipayung   
3            3                               toko dita, kertosono   
4            4                                      jl. orde baru   
...        ...                                                ...   
299995  299995               jend ahmad yani 331 kertasari ciamis   
299996  299996                 raya cila kko, cilandak timur kel.   
299997  299997                     tanjung gusta jl. yaya 2 no 17   
299998  299998  jalan cipadu jaya taman asri gang bijaksana 3 ...   
299999  299999          jaya maspion permata blok beryl b2  no.58   

                                       POI/street  
0       /jl kapuk timur delta sili iii lippo cika  
1                                               /  
2              

In [49]:
df_test = pd.DataFrame(test, columns= ['id','raw_address'])
print(df_test)

          id                                        raw_address
0          0              s. par 53 sidanegara 4 cilacap tengah
1          1              angg per, baloi indah kel. lubuk baja
2          2                              asma laun, mand imog,
3          3      ud agung rej, raya nga sri wedari karanganyar
4          4                         cut mutia, 35 baiturrahman
...      ...                                                ...
49995  49995                    toko mbak farid semboro semboro
49996  49996     vie - tk. ridho kids, vete 3 cari, 16720 ciawi
49997  49997                mart dan roti bakar malabar, nasio,
49998  49998  graha indah pamulang jl. mujair raya bambu apu...
49999  49999                                               adi,

[50000 rows x 2 columns]


In [7]:
#Call all the raw addresses
df_train.raw_address

0         jl kapuk timur delta sili iii lippo cika 11 a ...
1                                        aye, jati sampurna
2                      setu siung 119 rt 5 1 13880 cipayung
3                                      toko dita, kertosono
4                                             jl. orde baru
                                ...                        
299995                 jend ahmad yani 331 kertasari ciamis
299996                   raya cila kko, cilandak timur kel.
299997                       tanjung gusta jl. yaya 2 no 17
299998    jalan cipadu jaya taman asri gang bijaksana 3 ...
299999            jaya maspion permata blok beryl b2  no.58
Name: raw_address, Length: 300000, dtype: object

In [8]:
#Get a rows of addresses
df_train.loc[2:6]

,id,raw_address
2,2,setu siung 119 rt 5 1 13880 cipayung
3,3,"toko dita, kertosono"
4,4,jl. orde baru
5,5,"raya samb gede, 299 toko bb kids"
6,6,"kem mel raya, no 4 bojong rawalumbu rt 1 36 ra..."


In [9]:
df_train.raw_address.shape

(300000,)

## Tokenization

In [26]:
def tokenize(contents):
  allAddresses = contents.raw_address
  tokens = []
  for address in allAddresses:
    #print(address.split(", "))
    for text in address.split(", "):
      tokens.append(text)
  return tokens

In [29]:
#Get Tokens
train_tokens = tokenize(df_train)

In [31]:
#Number of tokens found
len(train_tokens)

474418

In [45]:
# number of UNIQUE tokens found
vocabulary = set(train_tokens)
len(vocabulary)

377844

In [47]:
frequency_dist = nltk.FreqDist(train_tokens)
#Show top 50 tokens
sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50]

['jend sudi',
 'ahmad yani',
 'jend ahmad yani',
 'pem',
 'gajah mada',
 'soek hatta',
 'vete',
 'yos suda',
 'imam bon',
 'dip',
 'gatot subr',
 'depok',
 'pahl',
 'teuku umar',
 'man',
 'no 1',
 'hayam wuruk',
 'pram',
 'brig kata',
 'jend sudi,',
 'merd',
 'kuta',
 'rw 1',
 'denpasar barat',
 'cengkareng',
 'setia budi',
 'ir. h. jua',
 'pang sudi',
 'kebon jeruk',
 'rw 4',
 'nasio',
 'ban',
 'jawa barat',
 'ahmad yani,',
 'mawar',
 'mas',
 'jati',
 'gar',
 'raya ser',
 'toko kelon',
 'taman sari',
 'peri kem',
 'denpasar selatan',
 'siliw',
 'urip sumoh',
 'waru',
 'tangerang',
 'cipayung',
 'bas rah',
 'balikpapan selatan']

## Set Up the Model

In [79]:
raws = df_train['raw_address']
results = df_train['POI/street'].to_numpy()

In [80]:
# TOKENIZE TEXT SEQUENCE
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tokens)
sequences = tokenizer.texts_to_sequences(raws)

# convert to numpy array
sequences = np.array(sequences)
x = sequences

In [81]:
# TOKENIZE TEXT SEQUENCE
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tokens)
sequences = tokenizer.texts_to_sequences(results)

# convert to numpy array
sequences = np.array(sequences)
y = sequences

In [82]:
print("X: ",x.shape)
print("Y: ",y.shape)

X:  (300000,)
Y:  (300000,)


In [83]:
print("Raw address: ",x[0])
print("POI/Street: ",y[0])

Raw address:  [8, 282, 12, 876, 7664, 39, 2147, 656, 31, 45, 6855, 107, 291]
POI/Street:  [8, 282, 12, 876, 7664, 39, 2147, 656]


In [84]:
#Variables
RANDOM_STATE = 50
EPOCHS = 150
BATCH_SIZE = 2048
TRAINING_LENGTH = 50
TRAIN_FRACTION = 0.7
LSTM_CELLS = 64
VERBOSE = 0
SAVE_MODEL = True